In [1]:
# Import modules
import pandas as pd
import json
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import preprocessor as p
import nltk
from nltk.tokenize import word_tokenize
import string
from collections import defaultdict
import contractions
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 500)


In [30]:
train_df = pd.read_csv('../res/preprocessed/train_final.csv')
test_df = pd.read_csv('../res/preprocessed/test_final.csv')
val_df = pd.read_csv('../res/preprocessed/val_final.csv')

In [31]:
train_df.head()


,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None
0,you just do not get it next time you are relyi...,0,0,1,1,0,0,0,1,0
1,it takes a certain kind of stupid to believe s...,0,1,0,1,0,0,1,0,0
2,the looney left liberal sjw cuck a sad delusio...,1,0,0,1,0,0,0,1,0
3,blacks are only of the yous population therefo...,0,0,1,0,1,0,0,0,0
4,its the islamic signal they are ready and will...,1,0,0,1,0,1,0,0,0


In [32]:
train_df.insert(5,"NOTPRFN", train_df['PRFN']) 
train_df['NOTPRFN'] = train_df['NOTPRFN'].apply(lambda x: 1 if x==0 else 0)

test_df.insert(5,"NOTPRFN", test_df['PRFN']) 
test_df['NOTPRFN'] = test_df['NOTPRFN'].apply(lambda x: 1 if x==0 else 0)

val_df.insert(5,"NOTPRFN", val_df['PRFN']) 
val_df['NOTPRFN'] = val_df['NOTPRFN'].apply(lambda x: 1 if x==0 else 0)



In [33]:
train_df.to_csv('../res/preprocessed/train_final.csv', index=False)
test_df.to_csv('../res/preprocessed/test_final.csv', index=False)
val_df.to_csv('../res/preprocessed/val_final.csv', index=False)


In [34]:
train_df.head()

,text,HATE,NOT,OFFN,PRFN,NOTPRFN,Race,Religion,Gender,Other,None
0,you just do not get it next time you are relyi...,0,0,1,1,0,0,0,0,1,0
1,it takes a certain kind of stupid to believe s...,0,1,0,1,0,0,0,1,0,0
2,the looney left liberal sjw cuck a sad delusio...,1,0,0,1,0,0,0,0,1,0
3,blacks are only of the yous population therefo...,0,0,1,0,1,1,0,0,0,0
4,its the islamic signal they are ready and will...,1,0,0,1,0,0,1,0,0,0


In [2]:
# df = pd.read_csv('../res/preprocessed/train_final.csv')
# for col in df.columns[1:]:
#    print(col)
#    print(df[col].value_counts())
#    print('----')


In [2]:
def clean_tweets_text(df, task_name):

    # Clean text with twitter-preprocessor
    clean = []
    for i, v in enumerate(df['text']):
        clean.append(p.clean(v))
    df["text"] = clean

    # Convert to lower case
    df["text"] = [entry.lower() for entry in df["text"]]

    # Expand Contractions


    def expand_contractions(s):
        expanded = []
        for word in s.split():
            expanded.append(contractions.fix(word))
        return(' '.join(expanded))


    df["text"] = [expand_contractions(entry) for entry in df['text']]

    # Remove punctuation marks
    df["text"] = [entry.translate(str.maketrans(
        '', '', string.punctuation)) for entry in df["text"]]

    # Tokenization
    # df['text']= [word_tokenize(entry) for entry in df['text']]
    df.to_csv(f'../res/preprocessed/{task_name}/{task_name}.csv', index=False)


### CLEAN task 1, 2 and 3 texts

In [3]:
task_df = pd.read_csv('../res/raw/subtask2.csv')
task_df.drop(["tweet_id",  "ID", "task2"], axis=1, inplace=True)

# Apply one hot encoding
task_df = task_df.join(pd.get_dummies(task_df['task1']))
task_df = task_df.dropna().drop(columns=['task1'])
task1_df = task_df
clean_tweets_text(task_df, 'task1')

In [4]:
task_df = pd.read_csv('../res/raw/subtask2.csv')
task_df.drop(["tweet_id",  "ID", "task1"], axis=1, inplace=True)

# Apply one hot encoding
task_df = task_df.join(pd.get_dummies(task_df['task2']))
task_df = task_df.dropna().drop(columns=['task2'])
task2_df = task_df
clean_tweets_text(task_df, 'task2')

In [5]:
df = pd.read_csv('../res/raw/raw_task3.csv')
df = df.rename(columns={'tweet_text':'text'})
clean_tweets_text(df, 'task3')

### Combine task1 and task2 one hot encoded

In [7]:
# df = pd.merge(task1_df, task2_df, left_index=True, right_index=True)
from sklearn.model_selection import train_test_split

task2_df_temp = task2_df.drop(columns=['text','HATE'])
df = pd.concat([task1_df, task2_df_temp], axis=1)
train_task1_2_df, test_task1_2_df = train_test_split(df, test_size=0.3)
val_task1_2_df, test_task1_2_df = train_test_split(test_task1_2_df, test_size=0.5)

train_task1_2_df.to_csv('../res/preprocessed/task1_2/train_task1_2.csv', index=False)

test_task1_2_df.to_csv('../res/preprocessed/task1_2/test_task1_2.csv', index=False)

val_task1_2_df.to_csv('../res/preprocessed/task1_2/val_task1_2.csv', index=False)
